In [1]:
import pandas as pd
import numpy as np

from statsmodels.tsa.stattools import pacf, acf

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.metrics import mean_absolute_error

# Plot functions

In [2]:
def create_corr_plot(series, plot_pacf=False):
    corr_array = pacf(series.dropna(), alpha=0.05) if plot_pacf else acf(series.dropna(), alpha=0.05)
    lower_y = corr_array[1][:,0] - corr_array[0]
    upper_y = corr_array[1][:,1] - corr_array[0]

    fig = go.Figure()
    [fig.add_scatter(x=(x,x), y=(0,corr_array[0][x]), mode='lines',line_color='#3f3f3f') 
     for x in range(len(corr_array[0]))]
    fig.add_scatter(x=np.arange(len(corr_array[0])), y=corr_array[0], mode='markers', marker_color='#1f77b4',
                   marker_size=12)
    fig.add_scatter(x=np.arange(len(corr_array[0])), y=upper_y, mode='lines', line_color='rgba(255,255,255,0)')
    fig.add_scatter(x=np.arange(len(corr_array[0])), y=lower_y, mode='lines',fillcolor='rgba(32, 146, 230,0.3)',
            fill='tonexty', line_color='rgba(255,255,255,0)')
    fig.update_traces(showlegend=False)
    fig.update_xaxes(range=[-1,20])
    fig.update_yaxes(zerolinecolor='#000000')
    
    title='Partial Autocorrelation (PACF)' if plot_pacf else 'Autocorrelation (ACF)'
    fig.update_layout(title=title)
    fig.show()

In [3]:
def plot_evolution_per_month(data):
    fig = go.Figure()
    color = ["#EBAFB9", "#C98F99", "#A76F79", "#865058", "#643038", "#421018"]
    quantity = data.iloc[:,START_TRAIN_INDEX:(END_TEST_INDEX+1)].sum().reset_index(name="Quantity").rename({"index" : "Date"}, axis=1)
    quantity["Date"] = pd.to_datetime(quantity["Date"])
    quantity["Year"] = quantity["Date"].dt.strftime('%Y')
    quantity["Month"] = quantity["Date"].dt.strftime('%B')
    for i, year in enumerate(range(2018, 2024)):
        fig.add_trace(go.Bar(
            x=quantity[quantity["Year"] == str(year)]["Month"],
            y=quantity[quantity["Year"] == str(year)]["Quantity"],
            name=year,
            marker_color=color[i]
        )
        )
    # Here we modify the tickangle of the xaxis, resulting in rotated labels.
    fig.update_layout(barmode='group', title = "Evolution per month")
    fig.update_xaxes(categoryorder='array', categoryarray= ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'])
    fig.show()